In [1]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from pymongo import MongoClient
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
# Creating a connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['examen2']
collection = db['senado']

In [3]:
# Base URL of the site to be analyzed
SITE_URL = 'https://alianzaverde.org.co'

In [4]:
# Firefox web driver path
DRIVER_PATH = './geckodriver'

In [5]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

C:\Users\eveli\AppData\Local\Temp\ipykernel_23960\1910320475.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [6]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(5)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [7]:
# Getting HTML content for a particular news listing page
soup = make_request(browser, '/lideres-verdes/congreso-2022/senado-2022')

In [8]:
# Finding the section where news are contained 
layout = soup.find(class_ = 'main-body')

In [9]:
# Getting blocks from layout
blocks = layout.find_all(class_ = 'sppb-col-md-3', recursive = True)
print(len(blocks))

36


In [10]:
# Building a list with title and relative path of the news founded

page = []
for card in blocks:
    page.append({
        'photo': card.find('div', class_ = 'sppb-person-image').find('img')['src'],
        'descrip': card.find('span', class_ = 'sppb-person-designation').get_text(),
        'red': card.find('ul', class_ = 'sppb-person-social').find('a')['href']
    })

AttributeError: 'NoneType' object has no attribute 'find'

In [11]:
page

[{'photo': '/images/Candidatos_2022/senado/2.jpg',
  'descrip': 'Abogada y filósofa, con un máster en Ciencias de la Administración Pública, en Alemania. Experta en gestión pública, descentralización, transparencia, buen gobierno, modernización del estado, cultura ciudadana y cooperación internacional, temas en los cuales se ha desempeñado alternativamente como servidora pública, consultora y docente.',
  'red': 'https://twitter.com/VivianaBarbere '},
 {'photo': '/images/Candidatos_2022/senado/3.jpg',
  'descrip': 'Nacida de seres excepcionales, en tiempos inciertos; es la mujer de retos resueltos. Desde el inicio de su carrera política, su liderazgo se enfoca en mantener la cultura de la equidad, la familia y el fortalecimiento económico, político y social de las regiones en Colombia',
  'red': 'https://www.facebook.com/sandraortizsenadora'},
 {'photo': '/images/Candidatos_2022/senado/5.jpg',
  'descrip': 'Destacado Senador de la República en el año 2010 fue disciplinado y estricto en

In [12]:
# Storing extracted information for further analysis
collection.insert_many(page)